# Ensemble : algoritmo Gradient Boosting

O objetivo é avaliar se um nódulo de mama é maligno ou benigno. Trata-se de dados de classificação de câncer de mama.



Modelo de Machine Learning para detecção do câncer de mama

Fonte: UCI ML Breast Cancer Wisconsin (Diagnostic) dataset is downloaded from: https://goo.gl/U2Uwz2

Os dados também estão disponibilizados pela própria biblioteca, que possui um pacote de Datasets. 

Ten real-valued features are computed for each cell nucleus:

a) radius (mean of distances from center to points on the perimeter)
b) texture (standard deviation of gray-scale values)
c) perimeter
d) area
e) smoothness (local variation in radius lengths)
f) compactness (perimeter^2 / area - 1.0)
g) concavity (severity of concave portions of the contour)
h) concave points (number of concave portions of the contour)
i) symmetry
j) fractal dimension ("coastline approximation" - 1)


Algoritmo Gradient Boosting em um modelo de classificação.

De acordo com a documentação do Scikit-Learn, os parâmetros mais importantes na concepção do modelo são:
learning_rate: Taxa de aprendizado que determina a importância de cada árvore na concepção do modelo final e na minimização do resíduo gerado. Deve ser um valor no intervalo de 0.0 a 1.
n_estimators: Número de árvores ou estágios utilizados na construção e treinamento do modelo.
Também é possível controlar o tamanho de cada árvore através dos parâmetros:
max_depth: A profundidade máxima da árvore.
max_leaf_nodes: Número máximo de folhas.

Existem outros parâmetros que podem ser observados na documentação oficial.

Vamos criar um modelo simples com alguns desses parâmetros definidos de forma default, pelo próprio algoritmo:


In [ ]:
# Importar as bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split



In [ ]:
# importar o dataset em csv
df = pd.read_csv(breast-cancer-wisconsin.csv)
# eliminar uma coluna com erro
df.drop('Unnamed: 32', axis=1, inplace=True)

 

In [ ]:
df.info()

Com exceção da coluna diagnosis, todas as outras são do tipo numérica (int e float). Apesar dessa ser a nossa conclusão olhando as primeiras entradas, é prudente analisar por meio do atributo dtypes para ter certeza que nenhuma foi importada como string.

In [ ]:
# dimensões do df
print("DIMENSÕES DO DATAFRAME:")
print("Linhas:\t\t{}".format(df.shape[0]))
print("Colunas:\t{}".format(df.shape[1]))

In [ ]:
df.head().T

In [ ]:
df.describe().T

In [ ]:
# Transformando a variável target

#opção 1
df['target'] = df['diagnosis'].map({'B': 0, 'M': 1}).astype(int)
df.head()



In [ ]:

df.drop(['id','diagnosis'], axis=1, inplace=True)

In [ ]:
correlation = df.corr()
plt.figure(figsize=(10,10))
sns.heatmap(correlation, vmax=1, square=True,annot=True,cmap='viridis')

plt.title('Correlation between different fearures')

In [ ]:
import matplotlib.pyplot as plt  
sns.boxplot(x='target', y='perimeter_worst', data=df)
plt.title('perimeter_worst x diagnosis')
plt.show()

In [ ]:
# Análise exploratória dos dados usando o gráfico Box Plot
features = df
fig,axs=plt.subplots(nrows=6, ncols=5, figsize=(18,10))
for col, ax in zip(features[0:], axs.ravel()):
    x=df.loc[:, col]
    sns.boxplot(x, ax=ax, orient='v')
    plt.subplots_adjust(top=0.92,bottom=0.08, left=0.10,right=0.95,hspace=0.25,wspace=0.4);

In [ ]:
# Análise exploratória dos dados usando o gráfico histograma

features = df
fig,axs=plt.subplots(nrows=5, ncols=6, figsize=(18,10))
for col, ax in zip(features[0:], axs.ravel()):
    x=df.loc[:, col]
    sns.distplot(x, ax=ax, color="blue", kde=False)
    plt.subplots_adjust(top=0.92,bottom=0.08, left=0.10,right=0.95,hspace=0.25,wspace=0.4)

In [ ]:
from sklearn.model_selection import train_test_split
# separar as variáveis independentes da variável alvo
X = df.drop(['target'], axis=1)
y = df['target']

In [ ]:
X

In [ ]:
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1),df['target'],test_size=0.30, random_state=17)


In [ ]:
X_train

Documentação sobre XGBoost https://xgboost.readthedocs.io/en/stable/index.html
    
class sklearn.ensemble.GradientBoostingClassifier(*, loss='log_loss', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, init=None, random_state=None, max_features=None, verbose=0, max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)

In [ ]:
model = GradientBoostingClassifier(random_state=17)


In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_predito = model.predict(X_test)


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_predito)

In [ ]:
print('Acurácia %0f' % model.score(X_test, y_test))


Podemos analisar o comportamento do algoritmo em cada etapa de ajuste através de um método chamado staged_predict.
Uma característica interessante desse método é que ele possibilita que façamos análises de desempenho que poderiam ser realizadas somente no modelo final, como medir a acurácia ou o erro médio do modelo. 
Essas análises são importantes, pois, pode nos auxiliar a determinar melhores parâmetros para o modelo.
A taxa de aprendizado determina a importância de cada árvore na concepção do modelo final e na minimização do resíduo gerado. Agora, utilizando o método staged_predict, vamos analisar o comportamento do algoritmo com alguns valores de learning_rate e analisar qual estágio o algoritmo irá atingir a minimização máxima do resíduo:


Segundo a documentação do algoritmo, existe uma troca entre os parâmetros n_estimators e learning_rate é evidente pelo gráfico gerado pelo algoritmo, que, em modelos treinados com uma taxa de aprendizado muito pequena, são necessários mais estágios para se chegar ao resíduo mínimo.


É recomendado a utilização de valores pequenos para o parâmetro learning_rate, visando manter a estabilidade do erro. 
Os valores recomendados são menores ou igual a 0.1, valor default definido pela biblioteca.
Se no exemplo anterior conseguimos identificar em qual estágio obtivemos o menor resíduo para cada taxa de aprendizado, então também podemos utilizar esse método para obter o número de ajustes ideal para o nosso modelo. 
Treinamos nosso modelo com 200 árvores com a taxa de 0.1 e limitamos a profundidade da árvore a 2. 

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np



In [ ]:
model = GradientBoostingClassifier(max_depth=2, n_estimators=200, learning_rate=0.1)
model.fit(X_train, y_train)
errors = [mean_squared_error(y_test, y_pred) for y_pred in model.staged_predict(X_test)]



In [ ]:
plt.plot(errors)
plt.ylabel('Resíduo')
plt.xlabel('Número de estágios/árvores')
plt.show()
print('Acurácia %0f' % model.score(X_test, y_test))

In [ ]:
best_n_estimators = np.argmin(errors) + 1
best_model = GradientBoostingClassifier(max_depth=2, n_estimators=best_n_estimators)
best_model.fit(X_train, y_train)
errors_best_model = [mean_squared_error(y_test, y_pred) for y_pred in best_model.staged_predict(X_test)]



In [ ]:
best_model

In [ ]:
plt.plot(errors_best_model)
plt.ylabel('Resíduo')
plt.xlabel('Número de estágios/árvores')
plt.show()
print('Acurácia %0f' % best_model.score(X_test, y_test))


Precisamos de apenas seis estágios de ajustes para melhorar nosso modelo, a acurácia agora foi de 96.27%, uma diferença pequena mas que pode ser um diferencial em problemas mais robustos.


O número de estágios que definimos no início funcionará como um número máximo de estágios e o nosso algoritmo sempre decidirá qual é o número ideal de estágios para uma melhor performance do nosso modelo.


In [ ]:
arvores = []
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]

In [ ]:
for learning_rate in learning_rates:
    # Declara o modelo
    model = GradientBoostingClassifier(max_depth=2, n_estimators=1000, learning_rate=learning_rate)
    # Treina o modelo
    model.fit(X_train, y_train)
    # Obtem os erros/residuos encontrados em cada estagio do algoritmo
    errors = [mean_squared_error(y_test, y_pred) for y_pred in model.staged_predict(X_test)]
    best_number_of_estimators = np.argmin(errors)
    arvores.append(best_number_of_estimators + 1)



In [ ]:
plt.plot(learning_rates, arvores)
plt.ylabel('Numero de arvores/estágios')
plt.xlabel('Taxa de aprendizado')
plt.show()

In [ ]:
print('Acurácia %0f' % best_model.score(X_test, y_test))


In [ ]:
y_pred = best_model.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix

# Matriz de Confusão

confusion_matrix = confusion_matrix(y_test, y_pred)
confusion_matrix

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

# Resultado do classification_report:

Light Boosting
https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html

In [ ]:
# installing LightGBM (Required in Jupyter Notebook and 
# few other compilers once)

!pip install lightgbm
  
# Importing Required Library
import pandas as pd
import lightgbm as lgb
  
# Similarly LGBMRegressor can also be imported for a regression model.
from lightgbm import LGBMClassifier
  


In [ ]:
  
# Creating an object for model and fitting it on training data set 
model = LGBMClassifier()
model.fit(X_train, y_train)

LightGBM classifier.

__init__(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=None, importance_type='split', **kwargs)

Alguns parâmetros importantes::

max_depth: define um limite para a profundidade da árvore. O valor padrão é 20. É eficaz no controle do ajuste.
categorical_feature: especifica o recurso categórico usado para o modelo de treinamento. 
bagging_fraction: especifica a fração de dados a ser considerada para cada iteração.
num_iterations: especifica o número de iterações a serem realizadas. O valor padrão é 100.
num_leaves: especifica o número de folhas em uma árvore. Deve ser menor que o quadrado de max_depth .
max_bin: especifica o número máximo de caixas para armazenar os valores do recurso.
min_data_in_bin: especifica a quantidade mínima de dados em um bin.
feature_fraction : especifica a fração de recursos a serem considerados em cada iteração. O valor padrão é um.

In [ ]:
  
# Predicting the Target variable
pred = model.predict(X_test)
print(pred)


In [ ]:
accuracy = model.score(X_test, y_test)
print(accuracy)

Ajuste de parâmetro
Alguns parâmetros importantes e seu uso estão listados abaixo:

max_depth: define um limite para a profundidade da árvore. O valor padrão é 20. É eficaz no controle do ajuste.
categorical_feature: especifica o recurso categórico usado para o modelo de treinamento. 
bagging_fraction: especifica a fração de dados a ser considerada para cada iteração.
num_iterations: especifica o número de iterações a serem realizadas. O valor padrão é 100.
num_leaves: especifica o número de folhas em uma árvore. Deve ser menor que o quadrado de max_depth .
max_bin: especifica o número máximo de caixas para armazenar os valores do recurso.
min_data_in_bin: especifica a quantidade mínima de dados em um bin.
tarefa: especifica a tarefa que desejamos realizar, que é treinamento ou previsão. A entrada padrão é train . Outro valor possível para este parâmetro é a previsão.
feature_fraction : especifica a fração de recursos a serem considerados em cada iteração. O valor padrão é um.

XGBoost (Extreme Gradient Boosting)

In [ ]:
# Documentação sobre XGBoost https://xgboost.readthedocs.io/en/stable/index.html

!pip install  xgboost

In [ ]:
import warnings
warnings.simplefilter('ignore')
# Import XGBoost
import xgboost
# XGBoost Classifier
from xgboost import XGBClassifier

In [ ]:
xgb = xgboost.XGBClassifier(n_estimators=500, max_depth=5, learning_rate=0.01, n_jobs=-1)
bst = xgb.fit(X_train, y_train)


In [ ]:
y_predito = xgb.predict(X_test)
accuracy_score(y_test, y_predito)

In [ ]:
from sklearn.metrics import confusion_matrix


# Matriz de Confusão

confusion_matrix = confusion_matrix(y_test, y_predito)
confusion_matrix

In [ ]:

from sklearn.metrics import ConfusionMatrixDisplay
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix, display_labels=['0', '1'])
disp.plot();


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_predito))

# Resultado do classification_report:

In [ ]:
from xgboost import plot_importance
    
# Plot feature importance
plot_importance(xgb);

In [ ]:
feature_important = xgb.get_booster().get_score(importance_type='weight')
keys = list(feature_important.keys())
values = list(feature_important.values())



In [ ]:
data = pd.DataFrame(data=values, index=keys, columns=["score"]).sort_values(by = "score", ascending=False)
data.nlargest(40, columns="score").plot(kind='barh', figsize = (20,10)) ## plot top 40 features

Usando RandomForest

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# separar as variáveis independentes da variável alvo
X = df.drop(['target'], axis=1)
y = df['target']


# padronizar as colunas numéricas
X= StandardScaler().fit_transform(X)
# label encoder na variável alvo
y= LabelEncoder().fit_transform(y)

# dividir o dataset entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)


In [ ]:
y

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# instanciando o modelo de Random Forest
rf_model = RandomForestClassifier(n_estimators = 10,max_depth=3, criterion='entropy', random_state = 17)
# treinando o modelo
rf_model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
# realizar as previsões no dataset de teste
y_pred = rf_model.predict(X_test)
# ver acurácia geral
print('[Acurácia] Random Forest:', accuracy_score(y_test, y_pred))
# imprimir o classification report
print('\n[Classification Report] Random Forest')
print( classification_report(y_test, y_pred))

In [ ]:
# plotar a matriz de confusão
pd.DataFrame(confusion_matrix(y_test, y_pred),
             index=['neg', 'pos'], columns=['pred_neg', 'pred_pos'])
